In [4]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
import os

# Carregar o dataset


file_path = f"{os.getcwd()}/data/Date_Fruit_Datasets/Date_Fruit_Datasets.xlsx"
data = pd.read_excel(file_path)

# Pré-processamento
X = data.iloc[:, :-1].values  # Todas as colunas exceto a última
y = data.iloc[:, -1].astype('category').cat.codes.values  # Última coluna (alvo)

# Normalize dataframe
X = (X - X.mean()) / X.std()

# Divisão em treino e teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Dataset customizado
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Modelo de Rede Neural
class Net(nn.Module):
    def __init__(self, input_size, output_size):
        super(Net, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.Tanh(),
            nn.Linear(512, 1024),
            nn.Tanh(),
            nn.Linear(1024, 1024),
            nn.Tanh(),
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Linear(512, 128),
            nn.Tanh(),
            nn.Linear(128, output_size)
        )
    
    def forward(self, x):
        return self.fc(x)

# Inicialização do modelo
input_size = X.shape[1]
output_size = len(set(y))
model = Net(input_size, output_size)

# Configuração de treino
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Funções de Treinamento e Teste
def train_model(model, loader):
    model.train()
    for X_batch, y_batch in loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()

def test_model(model, loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for X_batch, y_batch in loader:
            output = model(X_batch)
            pred = output.argmax(dim=1)
            correct += pred.eq(y_batch).sum().item()
    accuracy = correct / len(loader.dataset) * 100
    return accuracy

# Treinamento
epochs = 40
best_acc = 0
best_epoch = 0
for epoch in range(epochs):
    train_model(model, train_loader)
    acc = test_model(model, test_loader)
    if acc > best_acc:
        best_acc = acc
        best_epoch = epoch + 1
        torch.save(model.state_dict(), 'model.pth')
    print(f"Epoch {epoch+1}/{epochs}, Test Accuracy: {acc:.2f}%")

print(f"Best Accuracy @ Epoch{best_epoch}: {best_acc:.2f}%")


Epoch 1/40, Test Accuracy: 62.59%
Epoch 2/40, Test Accuracy: 58.89%
Epoch 3/40, Test Accuracy: 67.78%
Epoch 4/40, Test Accuracy: 67.41%
Epoch 5/40, Test Accuracy: 67.04%
Epoch 6/40, Test Accuracy: 65.56%
Epoch 7/40, Test Accuracy: 65.19%
Epoch 8/40, Test Accuracy: 69.63%
Epoch 9/40, Test Accuracy: 61.85%
Epoch 10/40, Test Accuracy: 68.89%
Epoch 11/40, Test Accuracy: 65.93%
Epoch 12/40, Test Accuracy: 67.78%
Epoch 13/40, Test Accuracy: 64.07%
Epoch 14/40, Test Accuracy: 69.26%
Epoch 15/40, Test Accuracy: 62.96%
Epoch 16/40, Test Accuracy: 68.89%
Epoch 17/40, Test Accuracy: 70.37%
Epoch 18/40, Test Accuracy: 71.48%
Epoch 19/40, Test Accuracy: 71.48%
Epoch 20/40, Test Accuracy: 63.33%
Epoch 21/40, Test Accuracy: 67.04%
Epoch 22/40, Test Accuracy: 69.63%
Epoch 23/40, Test Accuracy: 66.67%
Epoch 24/40, Test Accuracy: 73.33%
Epoch 25/40, Test Accuracy: 72.59%
Epoch 26/40, Test Accuracy: 70.74%
Epoch 27/40, Test Accuracy: 67.78%
Epoch 28/40, Test Accuracy: 70.00%
Epoch 29/40, Test Accuracy: 7